### Importamos 

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os


### Variables de entorno


In [3]:
load_dotenv()
api_key = os.getenv("API_KEY")  

### Subimos el documento

In [4]:

def upload_pdf(url: str):        
    try:
        loader = PyPDFLoader(url)
        loader = loader.lazy_load()

        text = ""

        for page in loader: 
            text += page.page_content + "\n"

        return text
    except Exception as e:
        print(e)
        return []
    


## Text Splitter para separar todo el contenido de mi documento
### aumento del chunk size para la obtención de oraciones más largas
### Decremento del chunk_overlap para intentar hacerle perder el contexto al modelo, diviendo las frases importantes para que pierda la relación entre las palabras

- CharacterTextSplitter es más directo, divide el texto en trozos pequeños fijos.
- Aquí utilicé RecursiveCharacterTextSplitter que según la documentación es más inteligente y jerarquico, ideal para PDFs, libros o documentación

In [5]:

def text_splitter(text): 

    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 30
    )
    texts = text_splitter.create_documents([text])
    print(texts)
    return texts



# Defino el modelo que utilizaré

1. Nuevo modelo de embedding de ollama (mxbai-embed-large): Es un modelo de incrustación (embedding) de lenguaje de última generación, de código abierto, desarrollado por Mixedbread.ai. 
- Su función principal es transformar texto (palabras, frases o documentos) en representaciones numéricas de alta dimensión, conocidas como vectores. Estos vectores capturan el significado semántico y sintáctico del texto, lo que permite a los sistemas de IA comprender la relación y similitud entre diferentes fragmentos de texto. 

* Este modelo solicitó otra vector Store ya que el anterior tenia un soporte para 768 dimensiones y este tiene hasta 1024 dimensiones al tratarse de un modelo de embedding más grande. Entonces, ¿a qué me refiero con dimensiones? la cantidad de números con los que cuenta un vector creado por el embedding

In [6]:
embedding = OllamaEmbeddings(
    model = "mxbai-embed-large:latest"
)

### Creo mi base de datos vectorial donde se guardará mi embedding

In [7]:
def get_vector_store(name_collection: str): 
    
    vector_store = Chroma(
    collection_name= name_collection,
    embedding_function=embedding,
    persist_directory="./prueba_chroma"
)    
    return vector_store

### Creo el retrieval que devolverá la información en una busqueda de similitudes

In [8]:
def retrieval(input_user: str): 
    vector_store = get_vector_store("langchainPrueba")
    docs = vector_store.similarity_search(input_user)
    return docs

### Creamos el propt system para el modelo

In [9]:
prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder preguntas sobre Arquitectura de software y solo debes contestar si el contexto no está vacio.
    En caso de que no cuentes con la información solicitada responde "La pregunta excede mi conocimiento" y si te preguntan algo fuera del contexto principal responde "No estoy programado para eso".
    Utiliza siempre el contexto proporcionado para responder y también utiliza un lenguaje familiar y amigable, con carisma.
    contexto = {contexto}
    pregunta del usuario: {input_user}
""")

### Creamos la función de respuesta que nos comunicará con nuestro agende de IA

In [10]:
def response(input_user: str, contexto: str):
    llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.0-flash-lite",
    temperature= 0.5
)

    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content

### Utilizamos las funciones para cargar el doc, aplicarle el text_splitter y guardar esos datos como embedding en la base de datos vectorial

In [11]:
loader = upload_pdf("asd.pdf")
texts = text_splitter(loader)
vector_store = get_vector_store("langchainPrueba")

vector_store.add_documents(texts)

[Document(metadata={}, page_content='Introducción  a  RAG   La  generación  aumentada  por  recuperación  (RAG)  es  el  proceso  de'), Document(metadata={}, page_content='(RAG)  es  el  proceso  de  optimización  de  la  salida'), Document(metadata={}, page_content='de\n \nun\n \nmodelo\n \nde\n \nlenguaje\n \nde\n \ngran\n \ntamaño,\n \nde\n \nmodo\n \nque\n \nhaga\n \nreferencia\n \na'), Document(metadata={}, page_content='que\n \nhaga\n \nreferencia\n \na\n \nuna\n \nbase\n \nde\n \nconocimientos\n \nautorizada\n \nfuera\n \nde\n \nlos'), Document(metadata={}, page_content='fuera\n \nde\n \nlos\n \norígenes\n \nde\n \ndatos\n \nde\n \nentrenamiento\n \nantes\n \nde\n \ngenerar\n \nuna'), Document(metadata={}, page_content='antes\n \nde\n \ngenerar\n \nuna\n \nrespuesta.\n \nLos\n \nmodelos\n \nde\n \nlenguaje\n \nde\n \ngran\n \ntamaño'), Document(metadata={}, page_content='de\n \ngran\n \ntamaño\n \n(LLM)\n \nse\n \nentrenan\n \ncon\n \nvolúmenes\n \nde\n \ndatos\n \namplios\n \ny

['420e6f3b-fcae-46f3-81cf-038485919851',
 '6a7eb5bb-0aa8-4488-8c57-12588256623d',
 'b8848432-a3d0-45d6-9087-ae4d1e4b4a87',
 '4b17398c-9b52-49e0-b4f9-657ab50e9b71',
 '1e6458c0-1d70-4bc5-8458-0b8664193878',
 '6ae942d3-50c0-4e62-8dfe-dadcac79c8a0',
 'd8ee3049-c2dc-47d8-8f05-ef3dad3b3252',
 'e1fad50a-6bcf-4cc3-82cb-451fa7c3efad',
 '03dc3154-5d09-40e1-8c47-b6086b0375d9',
 'b028c627-9969-48de-9053-9450f6ff7b32',
 '41eca284-6ac9-4346-9f3c-5f80e1bf5231',
 '6c7e0e1b-939a-4918-ad73-a4424178e453',
 '8e7df610-5bb0-4782-8edf-5b71809661d0',
 '1434989b-e169-46ab-9363-47174beea3d8',
 '03143d11-4df2-4494-a18e-561eb760e98b',
 '9b412f12-b67a-4c9a-b41e-043d7269d1a7',
 '76ab9f44-d440-4f57-8fae-a8af1ce3808e',
 'aea41400-74a2-4f4b-a63d-c3039559adfd',
 '608e343f-2e2a-42da-81ec-ff0fcb94c8ae',
 'cb79b10a-8994-4f18-85cf-cf45ffbb5beb',
 'aca102d3-417d-4875-ad8b-ad147f6bf709',
 'a535763d-d6b0-4fd2-b223-249d37d7dc65',
 'e58e6eda-1bf0-41b4-b171-8358f2494a54',
 '49332e1e-8662-45f3-a034-02f488bb3e70',
 '70dd375b-01de-

### Ponemos a prueba nuestro RAG
ya que se disminuyo el chunck_size y el overlap nos damos cuenta luego de la primera respuesta ya pierde el contexto.

In [ ]:

for l in range(3):
    input_user = input("Human: ")
    print(input_user)

    docs = retrieval(input_user=input_user)
    print(docs)

    for chunk in response(input_user=input_user, contexto=docs):
        print(chunk, end="", flush=True)
    

hola
[Document(id='bccb4657-9245-48dd-9d33-94c6faf4fe53', metadata={}, page_content='from langchain_ollama  import OllamaEmbeddings  #  creamos  la  comunicacion  con  nuestro'), Document(id='b1885b0e-36d0-489e-aa10-d4c1af53056d', metadata={}, page_content='from langchain_ollama  import OllamaEmbeddings  #  creamos  la  comunicacion  con  nuestro'), Document(id='073c74da-e3c4-4174-b425-7c98927d9955', metadata={}, page_content='esa  pregunta"      """),      ("ai","{ai_msg}"),      ("human","{human_msg}")  ])'), Document(id='3612cef6-244c-49c2-94fe-f407af988809', metadata={}, page_content='esa  pregunta"      """),      ("ai","{ai_msg}"),      ("human","{human_msg}")  ])')]
¡Hola! ¿En qué puedo ayudarte hoy? 😊
que me puedes enseñar?
[Document(id='72c46248-f623-4561-b054-6d55a10630c5', metadata={}, page_content='en  Langchain  y  evalúa  su'), Document(id='9c5379ad-a6d0-4848-8930-183c6fc38019', metadata={}, page_content='en  Langchain  y  evalúa  su'), Document(id='191b8de8-1fb1-4e32-bbb

### Al cambiar el chunck_size y el overlap dio una respuesta más concisa, al disminuirlo daba respuestas malas, no tan significativas y comprensibles

In [13]:

for l in range(2):
    input_user = input("Human: ")
    print(input_user)

    docs = retrieval(input_user=input_user)
    print(docs)

    for chunk in response(input_user=input_user, contexto=docs):
        print(chunk, end=" ", flush=True)
    


[Document(id='08c11b6a-d857-4af3-a582-691a8777a3a8', metadata={}, page_content='a\n \nlos\n \nFM\n \npara\n \nobtener\n \ninformación\n \nespecífica\n \nde\n \nla\n \norganización\n \no\n \ndel'), Document(id='94a68a20-4b03-4f71-ab83-f7761b67f7b8', metadata={}, page_content='a\n \nlos\n \nFM\n \npara\n \nobtener\n \ninformación\n \nespecífica\n \nde\n \nla\n \norganización\n \no\n \ndel'), Document(id='191b8de8-1fb1-4e32-bbb2-5498a3a7d49f', metadata={}, page_content='entrenamiento  originales  para'), Document(id='a6bca68b-7e2a-4663-9740-097e687b1f88', metadata={}, page_content='entrenamiento  originales  para')]
¡ Hola! 👋 Parece que necesitas ayuda con algo relacionado con Arquitectura de Software. Dime , ¿en qué puedo ayudarte? 😊
  hola
[Document(id='bccb4657-9245-48dd-9d33-94c6faf4fe53', metadata={}, page_content='from langchain_ollama  import OllamaEmbeddings  #  creamos  la  comunicacion  con  nuestro'), Document(id='b1885b0e-36d0-489e-aa10-d4c1af53056d', metadata={}, page_conten